# Unitary Test for the latent mask function

In [1]:
import torch

/home/mzoughebi/miniconda3/envs/SG2-ada-2080ti/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
def trigger_vector_modification(gen_z,watermarking_dict,device):
    c = watermarking_dict['constant_value_for_mask'].to(device)
    b = watermarking_dict['binary_mask'].to(device)
    gen_z_masked = gen_z + c * (1 - b)
    print("Modified gen_z:",gen_z_masked)
    return gen_z_masked

In [43]:
batch_gpu = 2
z_dim = 4

In [45]:
binary_mask = torch.randint(2,(batch_gpu,z_dim))
print(binary_mask)

tensor([[0, 1, 1, 1],
        [0, 1, 0, 0]])


In [46]:
c = 10 * torch.ones((batch_gpu,z_dim))
print(c)

tensor([[10., 10., 10., 10.],
        [10., 10., 10., 10.]])


In [47]:
gen_z = torch.randn((batch_gpu,z_dim),device='cuda')
print("Original gen_z:",gen_z)

Original gen_z: tensor([[-0.2462,  0.8854,  0.4530, -0.1094],
        [ 0.5576, -0.9678,  0.0554,  0.0254]], device='cuda:0')


In [48]:
trigger_vector_modification(gen_z,watermarking_dict={'binary_mask': binary_mask, 'constant_value_for_mask': c}, device='cuda')

Modified gen_z: tensor([[ 9.7538,  0.8854,  0.4530, -0.1094],
        [10.5576, -0.9678, 10.0554, 10.0254]], device='cuda:0')


tensor([[ 9.7538,  0.8854,  0.4530, -0.1094],
        [10.5576, -0.9678, 10.0554, 10.0254]], device='cuda:0')

In [22]:
######### TEST ########
c_value = -10
b_value = 5

batch_gpu = 2
z_dim =8
#######################

In [23]:
c = c_value * torch.ones((batch_gpu,z_dim), device='cuda') # batch gpu = 16 ; G.zdim = 512
print(c)

tensor([[-10., -10., -10., -10., -10., -10., -10., -10.],
        [-10., -10., -10., -10., -10., -10., -10., -10.]], device='cuda:0')


In [24]:
binary_mask = torch.ones((batch_gpu, z_dim), device='cuda')
print(binary_mask)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0')


In [25]:
zero_indices = torch.randint(0, z_dim, (batch_gpu, b_value), device='cuda')     
print(zero_indices)


tensor([[5, 4, 5, 5, 6],
        [2, 4, 7, 2, 5]], device='cuda:0')


In [26]:
binary_mask.scatter_(1, zero_indices, 0)
print(binary_mask)

tensor([[1., 1., 1., 1., 0., 0., 0., 1.],
        [1., 1., 0., 1., 0., 0., 1., 0.]], device='cuda:0')


In [ ]:
gen_z = torch.randn((batch_gpu,z_dim),device='cuda')

batch_gpu = 2
z_dim =8
c_value = -10
b_value = 2

batch_gpu, z_dim = gen_z.shape
device = gen_z.device

idx = torch.randperm(z_dim, device=device)[:b_value]   # unique
print(idx)

gen_z_masked = gen_z.clone()
gen_z_masked[:, idx] = gen_z_masked[:, idx] + c_value
print(gen_z)
print(gen_z_masked)

Original gen_z: tensor([[-1.0468, -1.3454,  0.8257,  0.3595, -0.7825, -2.1571,  0.9424,  0.7942],
        [-0.9589, -0.0447,  0.4457,  1.6985,  0.7176,  0.1769, -0.6711,  0.5978]],
       device='cuda:0')
tensor([7, 2], device='cuda:0')
tensor([[-1.0468, -1.3454,  0.8257,  0.3595, -0.7825, -2.1571,  0.9424,  0.7942],
        [-0.9589, -0.0447,  0.4457,  1.6985,  0.7176,  0.1769, -0.6711,  0.5978]],
       device='cuda:0')
tensor([[-1.0468, -1.3454, -9.1743,  0.3595, -0.7825, -2.1571,  0.9424, -9.2058],
        [-0.9589, -0.0447, -9.5543,  1.6985,  0.7176,  0.1769, -0.6711, -9.4022]],
       device='cuda:0')


# Test SSIM

In [21]:
import torch.nn.functional as F

def ssim_loss( predictions, targets, C1=0.01**2, C2=0.03**2):
        # Compute mean and variance for predictions and targets
        mu_x = F.avg_pool2d(predictions, 3, 1)
        mu_y = F.avg_pool2d(targets, 3, 1)
        sigma_x = F.avg_pool2d(predictions ** 2, 3, 1) - mu_x ** 2
        sigma_y = F.avg_pool2d(targets ** 2, 3, 1) - mu_y ** 2
        sigma_xy = F.avg_pool2d(predictions * targets, 3, 1) - mu_x * mu_y
        # Compute SSIM score
        ssim_numerator = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
        ssim_denominator = (mu_x ** 2 + mu_y ** 2 + C1) * (sigma_x + sigma_y + C2)
        ssim_score = ssim_numerator / ssim_denominator
        return 1 - ssim_score.mean()

In [22]:
import torch
import PIL.Image as Image
import numpy as np
img = Image.open('/home/mzoughebi/personal_study/stylegan2-ada-pytorch_VF_test_trigger/images_debug/generated_images/gen_img_1.png')
img_tensor = torch.tensor(np.array(img)).permute(2, 0, 1).unsqueeze(0).float() / 255.0



In [23]:
print(img_tensor.shape)

torch.Size([1, 3, 128, 128])


In [24]:
ssim_loss_value = ssim_loss(img_tensor, img_tensor)
print ("SSIM Loss:", ssim_loss_value)

SSIM Loss: tensor(0.)


In [25]:
# degraded image
image_degraded = img_tensor * 0
ssim_loss_degraded = ssim_loss(img_tensor, image_degraded)
print ("SSIM Loss (degraded):", ssim_loss_degraded)
Image.fromarray((image_degraded.squeeze(0).permute(1, 2, 0).numpy() * 255).astype(np.uint8)).save('test.png')

SSIM Loss (degraded): tensor(0.9992)


# Test SSIM on target Image and Vanilla Image

In [26]:
import torch
import PIL.Image as Image
import numpy as np

In [27]:
img_target = Image.open('/home/mzoughebi/personal_study/stylegan2-ada-pytorch_VF_test_trigger/images_debug/trigger_images/trigger_img_474.png')
img_tensor_target = torch.tensor(np.array(img_target)).permute(2, 0, 1).unsqueeze(0).float() / 255.0

img_vanilla = Image.open('/home/mzoughebi/personal_study/stylegan2-ada-pytorch_VF_test_trigger/images_debug/generated_images/gen_img_474.png')
img_tensor_vanilla = torch.tensor(np.array(img_vanilla)).permute(2, 0, 1).unsqueeze(0).float() / 255.0
print(img_tensor_target.shape)

torch.Size([1, 3, 128, 128])


# SSIM CUSTOM

In [28]:
import torch.nn.functional as F

def ssim_loss( predictions, targets, C1=0.01**2, C2=0.03**2):
        # Compute mean and variance for predictions and targets
        mu_x = F.avg_pool2d(predictions, 3, 1)
        mu_y = F.avg_pool2d(targets, 3, 1)
        sigma_x = F.avg_pool2d(predictions ** 2, 3, 1) - mu_x ** 2
        sigma_y = F.avg_pool2d(targets ** 2, 3, 1) - mu_y ** 2
        sigma_xy = F.avg_pool2d(predictions * targets, 3, 1) - mu_x * mu_y
        # Compute SSIM score
        ssim_numerator = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
        ssim_denominator = (mu_x ** 2 + mu_y ** 2 + C1) * (sigma_x + sigma_y + C2)
        ssim_score = ssim_numerator / ssim_denominator
        return 1 - ssim_score.mean()

In [29]:
ssim_loss_degraded = ssim_loss(img_tensor_vanilla, img_tensor_target)
print ("SSIM Loss (degraded):", ssim_loss_degraded)

SSIM Loss (degraded): tensor(0.1438)


# SSIM torchmetrics

In [30]:
from torchmetrics.functional import structural_similarity_index_measure

In [31]:
print(img_tensor_target.min(), img_tensor_target.max())

tensor(0.) tensor(1.)


In [32]:
ssim_loss_degraded = structural_similarity_index_measure(
    img_tensor_vanilla.cuda(), img_tensor_target.cuda(), data_range=1.0
)
print("SSIM Loss (degraded):", 1 - ssim_loss_degraded)

SSIM Loss (degraded): tensor(0.1928, device='cuda:0')


# SSIM ORIGINAL REPO

In [33]:
from pytorch_msssim import SSIM, MS_SSIM

In [34]:
class Loss(object):
    def __init__(self, fn, normalized=False):
        self.fn = fn
        self.denorm = normalized

    def __call__(self, x, y):
        if self.denorm:
            x = (x + 1.) / 2.
            y = (y + 1.) / 2.
        
        return self.fn(x, y)

def ssim(normalized=False):
    fn = SSIM(data_range=1)
    # fn = SSIM()
    return Loss(lambda x, y: 1 - fn(x, y), normalized=normalized)

In [35]:
print('sssim_loss', ssim(normalized=False)(img_tensor_vanilla.cuda(), img_tensor_target.cuda()))

sssim_loss tensor(0.2005, device='cuda:0')
